In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
player = pd.read_csv('IPL_AUCTION_2013-2022.csv')
player.head()

,TID,PLAYER,NATIONALITY,TYPE,PRICE,SOLD/UNSOLD,TEAM,YEAR
0,2022-Mumbai Indians,Ishan Kishan,Indian,Wicket Keeper,"15,25,00,000",SOLD,Mumbai Indians,2022
1,2022-Chennai Super Kings,Deepak Chahar,Indian,Bowler,"14,00,00,000",SOLD,Chennai Super Kings,2022
2,2022-Kolkata Knight Riders,Shreyas Iyer,Indian,Batsman,"12,25,00,000",SOLD,Kolkata Knight Riders,2022
3,2022-Punjab Kings,Liam Livingstone,Overseas,All-Rounder,"11,50,00,000",SOLD,Punjab Kings,2022
4,2022-Delhi Capitals,Shardul Thakur,Indian,Bowler,"10,75,00,000",SOLD,Delhi Capitals,2022


In [3]:
player.drop(['TID','NATIONALITY','TEAM'],axis=1,inplace=True)
player.head()

,PLAYER,TYPE,PRICE,SOLD/UNSOLD,YEAR
0,Ishan Kishan,Wicket Keeper,"15,25,00,000",SOLD,2022
1,Deepak Chahar,Bowler,"14,00,00,000",SOLD,2022
2,Shreyas Iyer,Batsman,"12,25,00,000",SOLD,2022
3,Liam Livingstone,All-Rounder,"11,50,00,000",SOLD,2022
4,Shardul Thakur,Bowler,"10,75,00,000",SOLD,2022


In [4]:
player = player[player['SOLD/UNSOLD']=='SOLD']
player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 970 entries, 0 to 3837
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PLAYER       970 non-null    object
 1   TYPE         970 non-null    object
 2    PRICE       970 non-null    object
 3   SOLD/UNSOLD  970 non-null    object
 4   YEAR         970 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 45.5+ KB


In [5]:
df_no = []
df_batting = []
for name in tqdm(player['PLAYER']):
    try:
        url = 'https://search.espncricinfo.com/ci/content/site/search.html?search=' + name.lower().replace(' ','+') + '&x=0&y=0'
        page=requests.get(url)
        soup = BeautifulSoup(page.content,'html.parser')
        player_id = str(soup.find_all('h3')[0]).split('.html')[0].split('/')[-1]
        df = pd.read_html(f'https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class=11;template=results;type=batting;view=innings')[3]
        df['Name'] = name
        df_batting.append(df)
    except:
        df_no.append(df)


100%|██████████| 970/970 [23:30<00:00,  1.45s/it]


In [6]:
df_batting = pd.concat(df_batting)
df_no = pd.concat(df_no)

In [7]:
df_nb = []
df_bowling = []
for name in tqdm(player['PLAYER']):
    try:
        url = 'https://search.espncricinfo.com/ci/content/site/search.html?search=' + name.lower().replace(' ','+') + '&x=0&y=0'
        page=requests.get(url)
        soup = BeautifulSoup(page.content,'html.parser')
        player_id = str(soup.find_all('h3')[0]).split('.html')[0].split('/')[-1]
        df = pd.read_html(f'https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class=11;template=results;type=bowling;view=innings')[3]
        df['Name'] = name
        df_bowling.append(df)
    except:
        df_nb.append(df)


100%|██████████| 970/970 [23:21<00:00,  1.44s/it]


In [8]:
df_bowling = pd.concat(df_bowling)
df_nb = pd.concat(df_nb)

In [9]:
df_batting.to_csv('batting.csv',index=False)
df_bowling.to_csv('bowling.csv',index=False)
df_no.to_csv('notbat.csv',index=False)
df_nb.to_csv('notbowl.csv',index=False)